# Imports

In [15]:
# misc
import os

# plot
import matplotlib.pyplot as plt
from PIL import Image

# datascience libs
import numpy as np
import pandas as pd



try: # python
    path_ = os.path.join(os.path.dirname(os.path.abspath(__file__)), "..")
except NameError: # jupyter notebook
    path_ = os.path.dirname(os.getcwd())

dataset_dir = os.path.join(path_, "datasets")
model_dir = os.path.join(path_, "models")

# Read mapping file

In [41]:
def emnist_get_mapping(filepath):
    max_index = 0
    data = {}
    with open(filepath) as f:
        while True:
            line = f.readline()
            if not line:
                break
            tmp = [int(s) for s in line.strip().split(' ') if s.isdigit()]
            data[tmp[0]] = tmp[1]
            max_index = max(max_index, tmp[0])
    # create array with size
    emnist_mapping = [-1] * (max_index + 1)
    # dict to array
    for key, val in data.items():
        emnist_mapping[key] = val
    return emnist_mapping

set_name = 'emnist-balanced'
emnist_mapping = emnist_get_mapping(os.path.join(dataset_dir, set_name + '-mapping.txt'))

print(emnist_mapping)

[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 100, 101, 102, 103, 104, 110, 113, 114, 116]


# Converter

In [3]:
df = pd.read_csv(os.path.join(dataset_dir, 'emnist-balanced-test.csv'), header=None)

In [4]:
"""filepath = os.path.join(dataset_dir, 'emnist-balanced-test.csv')
with open(filepath) as f:
    line = f.readline()
print(line)"""

"filepath = os.path.join(dataset_dir, 'emnist-balanced-test.csv')\nwith open(filepath) as f:\n    line = f.readline()\nprint(line)"

In [46]:
def makedirs(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def write_image(raw_data, output_dir, i, X_set='train'):
    png_filename = os.path.join(output_dir, f"{X_set}_{i}.png")
    pixels = np.flip(np.array(raw_data, dtype='uint8').reshape(28,28), axis=1)
    #pixels = np.flip(pixels, axis=1)
    pixels = np.rot90(pixels, k=1, axes=(0, 1))
    #print(pixels, output_dir)
    img = Image.fromarray(pixels, mode='L')
    img.save(png_filename, format='PNG', bits=8)

def emnist_convert(dataset_dir, input_name, output_dir, emnist_mapping, X_set='train'):
    X_set_dir = os.path.join(dataset_dir, output_dir, X_set)
    makedirs(X_set_dir)
    
    input_file = os.path.join(dataset_dir, input_name+'-'+X_set+'.csv')
    i = 0
    with open(input_file) as f:
        while True:
            line = f.readline()
            if not line:
                break
            tmp = [int(s) for s in line.strip().split(',') if s.isdigit()]
            write_image(tmp[1:], X_set_dir, i, X_set)
            print(chr(emnist_mapping[tmp[0]]))
            i += 1
            break # temporary
    
set_name = 'emnist-balanced'
emnist_convert(dataset_dir, set_name, 'origin-'+set_name, emnist_mapping, 'test')


g
